In [1]:
pip install tensorflow pandas numpy matplotlib opencv-python scikit-learn kaggle

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
print(os.listdir("/kaggle/input/food-101/food-101/food-101/images/"))

['macarons', 'french_toast', 'lobster_bisque', 'prime_rib', 'pork_chop', 'guacamole', 'baby_back_ribs', 'mussels', 'beef_carpaccio', 'poutine', 'hot_and_sour_soup', 'seaweed_salad', 'foie_gras', 'dumplings', 'peking_duck', 'takoyaki', 'bibimbap', 'falafel', 'pulled_pork_sandwich', 'lobster_roll_sandwich', 'carrot_cake', 'beet_salad', 'panna_cotta', 'donuts', 'red_velvet_cake', 'grilled_cheese_sandwich', 'cannoli', 'spring_rolls', 'shrimp_and_grits', 'clam_chowder', 'omelette', 'fried_calamari', 'caprese_salad', 'oysters', 'scallops', 'ramen', 'grilled_salmon', 'croque_madame', 'filet_mignon', 'hamburger', 'spaghetti_carbonara', 'miso_soup', 'bread_pudding', 'lasagna', 'crab_cakes', 'cheesecake', 'spaghetti_bolognese', 'cup_cakes', 'creme_brulee', 'waffles', 'fish_and_chips', 'paella', 'macaroni_and_cheese', 'chocolate_mousse', 'ravioli', 'chicken_curry', 'caesar_salad', 'nachos', 'tiramisu', 'frozen_yogurt', 'ice_cream', 'risotto', 'club_sandwich', 'strawberry_shortcake', 'steak', 'chu

In [1]:
import os
import shutil
import random



# Define paths
base_dir = "/kaggle/input/food-101/food-101/food-101/images/"
output_dir = "/path/to/Food-101-split"
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Create train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split each folder into train and test
split_ratio = 0.8  # 80% training, 20% testing

for category in os.listdir(base_dir):
    category_path = os.path.join(base_dir, category)
    if os.path.isdir(category_path):  # Check if it's a directory
        # Get all image file paths
        images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
        random.shuffle(images)

        # Split into train and test
        split_point = int(len(images) * split_ratio)
        train_images, test_images = images[:split_point], images[split_point:]

        # Create category directories in train/test folders
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # Move files to train/test directories
        for img in train_images:
            shutil.copy(img, os.path.join(train_dir, category))
        for img in test_images:
            shutil.copy(img, os.path.join(test_dir, category))

print("Data successfully split into train and test directories!")
print(test_dir)
print(train_dir)

Data successfully split into train and test directories!
/path/to/Food-101-split/test
/path/to/Food-101-split/train


In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,965,669 (11.31 MB)

 Trainable params: 707,685 (2.70 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2710s 1s/step - accuracy: 0.3489 - loss: 2.7278 - val_accuracy: 0.4978 - val_loss: 1.9772
Epoch 2/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2759s 1s/step - accuracy: 0.4965 - loss: 1.9672 - val_accuracy: 0.5144 - val_loss: 1.9161
Epoch 3/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2680s 1s/step - accuracy: 0.5234 - loss: 1.8570 - val_accuracy: 0.5195 - val_loss: 1.8986
Epoch 4/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2728s 1s/step - accuracy: 0.5362 - loss: 1.7811 - val_accuracy: 0.5303 - val_loss: 1.8714
Epoch 5/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2726s 1s/step - accuracy: 0.5512 - loss: 1.7090 - val_accuracy: 0.5345 - val_loss: 1.8633
Epoch 6/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2695s 1s/step - accuracy: 0.5620 - loss: 1.6754 - val_accuracy: 0.5273 - val_loss: 1.9253
Epoch 7/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2643s 1s/step - accuracy: 0.5713 - loss: 1.6259 - val_accuracy: 0.5326 - val_loss: 1.8998
Epoch 8/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2689s 1s/step - accuracy: 0.5775 - loss: 1.60

In [6]:
import pandas as pd

calorie_mapping = {
    "macarons": 230,
    "french_toast": 300,
    "lobster_bisque": 160,
    "prime_rib": 350,
    "pork_chop": 240,
    "guacamole": 150,
    "baby_back_ribs": 300,
    "mussels": 172,
    "beef_carpaccio": 150,
    "poutine": 320,
    "hot_and_sour_soup": 70,
    "seaweed_salad": 200,
    "foie_gras": 400,
    "dumplings": 110,
    "peking_duck": 450,
    "takoyaki": 300,
    "bibimbap": 550,
    "falafel": 170,
    "pulled_pork_sandwich": 650,
    "lobster_roll_sandwich": 400,
    "carrot_cake": 300,
    "beet_salad": 150,
    "panna_cotta": 230,
    "donuts": 200,
    "red_velvet_cake": 350,
    "grilled_cheese_sandwich": 400,
    "cannoli": 300,
    "spring_rolls": 100,
    "shrimp_and_grits": 400,
    "clam_chowder": 180,
    "omelette": 150,
    "fried_calamari": 250,
    "caprese_salad": 200,
    "oysters": 90,
    "scallops": 150,
    "ramen": 350,
    "grilled_salmon": 220,
    "croque_madame": 500,
    "filet_mignon": 500,
    "hamburger": 250,
    "spaghetti_carbonara": 400,
    "miso_soup": 60,
    "bread_pudding": 350,
    "lasagna": 400,
    "crab_cakes": 180,
    "cheesecake": 400,
    "spaghetti_bolognese": 450,
    "cup_cakes": 200,
    "creme_brulee": 300,
    "waffles": 400,
    "fish_and_chips": 500,
    "paella": 400,
    "macaroni_and_cheese": 400,
    "chocolate_mousse": 350,
    "ravioli": 300,
    "chicken_curry": 250,
    "caesar_salad": 200,
    "nachos": 300,
    "tiramisu": 450,
    "frozen_yogurt": 100,
    "ice_cream": 200,
    "risotto": 350,
    "club_sandwich": 400,
    "strawberry_shortcake": 350,
    "steak": 700,
    "churros": 200,
    "garlic_bread": 180,
    "baklava": 300,
    "bruschetta": 150,
    "hummus": 180,
    "chicken_wings": 200,
    "greek_salad": 150,
    "tuna_tartare": 180,
    "chocolate_cake": 350,
    "gyoza": 220,
    "eggs_benedict": 450,
    "deviled_eggs": 180,
    "samosa": 150,
    "sushi": 200,
    "breakfast_burrito": 350,
    "ceviche": 150,
    "beef_tartare": 200,
    "apple_pie": 237,
    "huevos_rancheros": 300,
    "beignets": 250,
    "pizza": 266,
    "edamame": 120,
    "french_onion_soup": 150,
    "hot_dog": 150,
    "tacos": 150,
    "chicken_quesadilla": 350,
    "pho": 300,
    "gnocchi": 350,
    "pancakes": 250,
    "fried_rice": 300,
    "cheese_plate": 400,
    "onion_rings": 250,
    "escargots": 120,
    "sashimi": 180,
    "pad_thai": 350,
    "french_fries": 300
}


# Save as a DataFrame for reference
calorie_df = pd.DataFrame(list(calorie_mapping.items()), columns=["Food", "Calories_per_100g"])
print(calorie_df.head())


             Food  Calories_per_100g
0        macarons                230
1    french_toast                300
2  lobster_bisque                160
3       prime_rib                350
4       pork_chop                240


In [7]:
def estimate_calories(predicted_class, portion_size_grams):
    calories_per_100g = calorie_mapping.get(predicted_class, 0)
    return (calories_per_100g / 100) * portion_size_grams


In [10]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_and_estimate(image_path, portion_size_grams):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the food category
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = list(train_generator.class_indices.keys())[predicted_class_idx]

    # Estimate calories
    estimated_calories = estimate_calories(predicted_class, portion_size_grams)

    return predicted_class, estimated_calories

# Example usage
image_path = "/kaggle/input/cake-img/cake.jpeg"
portion_size_grams = 200  # Example portion size
food, calories = predict_and_estimate(image_path, portion_size_grams)
print(f"Predicted Food: {food}, Estimated Calories: {calories:.2f} kcal")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted Food: tiramisu, Estimated Calories: 900.00 kcal
